### Course lecture
[1 - Introduction to Random Forests](http://course18.fast.ai/lessonsml1/lesson1.html)

### Install packages

In [1]:
# Uncomment to install utilities: https://github.com/bogeholm/dataworks
#!pip install --upgrade git+git://github.com/bogeholm/dataworks.git

### Imports

In [2]:
import numpy as np
import pandas as pd

#from collections import OrderedDict
#from pandas.api.types import is_numeric_dtype
#from typing import Any, Iterable
from dataworks.df_utils import *#inspect_df, summarize_df, add_datefields, add_nan_columns, numeric_nans, categorize_df


pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [3]:
PATH = 'data/bulldozers/'

In [4]:
df_raw = pd.read_csv(f'{PATH}Train.zip', low_memory=False, parse_dates=['saledate'])

In [5]:
# Smaller DF to test algorithms
#dft = df_raw.iloc[0:50].copy(deep=True)

### Utility functions

In [6]:
def display_allrows(df):
    """ Override max rows and display them all
    """
    with pd.option_context('display.max_rows', len(df)):
            display(df)

### Inspect the data

In [7]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,...,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,...,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,...,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,...,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,...,NaN,NaN,NaN,NaN,NaN


In [8]:
inspect = inspect_df(df_raw)
#display_allrows(inspect)

In [9]:
summary = summarize_df(df_raw)
display_allrows(summary)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,datetime64[ns],1,0,0,401125,0.00
1,float64,2,0,0,802250,0.00
2,int64,6,0,0,2406750,0.00
3,object,44,22,6002766,17649500,0.34


### Add log price

In [10]:
df_proc = df_raw.copy(deep=True)
if 'SalePrice' in df_proc.columns:
    df_proc['LogSalePrice'] = np.log(df_proc['SalePrice'])
    df_proc.drop(columns=['SalePrice'], inplace=True)

In [11]:
df_proc.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,LogSalePrice
0,1139246,999089,3157,121,3.0,...,NaN,NaN,Standard,Conventional,11.097410
1,1139248,117657,77,121,3.0,...,NaN,NaN,Standard,Conventional,10.950807
2,1139249,434808,7009,121,3.0,...,NaN,NaN,NaN,NaN,9.210340
3,1139251,1026470,332,121,3.0,...,NaN,NaN,NaN,NaN,10.558414
4,1139253,1057373,17311,121,3.0,...,NaN,NaN,NaN,NaN,9.305651


### Extract date properties
See [Attributes of Pandas Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)

In [12]:
if 'saledate' in df_proc.columns:
    df_proc = add_datefields(df_raw, 'saledate', drop_original=True)

In [13]:
df_proc.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,saledate_is_month_start,saledate_is_quarter_end,saledate_is_quarter_start,saledate_quarter,saledate_week
0,1139246,66000,999089,3157,121,...,False,False,False,4,46
1,1139248,57000,117657,77,121,...,False,False,False,1,13
2,1139249,10000,434808,7009,121,...,False,False,False,1,9
3,1139251,38500,1026470,332,121,...,False,False,False,2,20
4,1139253,11000,1057373,17311,121,...,False,False,False,3,30


### Add NaN column
Here we add a columns indicating whether the original column contained a NaN

In [14]:
df_proc = add_nan_columns(df_proc)
df_proc.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_isnull,Blade_Type_isnull,Travel_Controls_isnull,Differential_Type_isnull,Steering_Controls_isnull
0,1139246,66000,999089,3157,121,...,True,True,True,False,False
1,1139248,57000,117657,77,121,...,True,True,True,False,False
2,1139249,10000,434808,7009,121,...,True,True,True,True,True
3,1139251,38500,1026470,332,121,...,True,True,True,True,True
4,1139253,11000,1057373,17311,121,...,True,True,True,True,True


### Handle numerical types containing NaN

In [15]:
stats = numeric_nans(df_raw)
stats

,column,null_fraction,nulls,type,is_numeric,num_uniques,uniques
0,auctioneerID,0.050,20136,float64,True,30,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,MachineHoursCurrentMeter,0.644,258360,float64,True,15152,"[0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."


In [16]:
df_proc['auctioneerID'] = df_proc['auctioneerID'].fillna(
    df_proc['auctioneerID'].max() + 1
)

df_proc['MachineHoursCurrentMeter'] = df_proc['MachineHoursCurrentMeter'].fillna(
    df_proc['MachineHoursCurrentMeter'].median()
)

In [17]:
numeric_nans(df_proc)

,column,null_fraction,nulls,type,is_numeric,num_uniques,uniques


### Add categories
- http://benalexkeen.com/mapping-categorical-data-in-pandas/
- https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

In [18]:
(df_cats, catcodes) = categorize_df(df_proc)

In [19]:
df_cats.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
0,1139246,66000,999089,3157,121,...,0,0,0,4,2
1,1139248,57000,117657,77,121,...,0,0,0,4,2
2,1139249,10000,434808,7009,121,...,0,0,0,0,0
3,1139251,38500,1026470,332,121,...,0,0,0,0,0
4,1139253,11000,1057373,17311,121,...,0,0,0,0,0


In [20]:
catcodes.head()

,column,n_categories,categories,codes
0,UsageBand,4,"Index(['High', 'Low', 'Medium'], dtype='object')",0 2 1 2 2 1 3 ...
1,fiModelDesc,4999,"Index(['100C', '104', '1066', '1066E', '1080',...",0 950 1 1725 2 331 3...
2,fiBaseModel,1950,"Index(['10', '100', '104', '1066', '1080', '10...",0 296 1 527 2 110 3...
3,fiSecondaryDesc,176,"Index([' MSR SPIN ACE', '#NAME?', '-2', '-3', ...",0 41 1 55 2 0 3 ...
4,fiModelSeries,123,"Index([' III', '#NAME?', '-1', '-1.50E+01', '-...",0 0 1 98 2 0 3 ...


### Inspect results of data processing

In [21]:
summarize_df(df_raw)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,datetime64[ns],1,0,0,401125,0.00
1,float64,2,0,0,802250,0.00
2,int64,6,0,0,2406750,0.00
3,object,44,22,6002766,17649500,0.34


In [22]:
summarize_df(df_cats)

,type,ncols,ncols_w_nans,n_nans,n_total,nan_frac
0,bool,44,0,0,17649500,0.0
1,float64,2,0,0,802250,0.0
2,int16,4,0,0,1604500,0.0
3,int64,10,0,0,4011250,0.0
4,int8,40,0,0,16045000,0.0


In [23]:
df_cats.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
0,1139246,66000,999089,3157,121,...,0,0,0,4,2
1,1139248,57000,117657,77,121,...,0,0,0,4,2
2,1139249,10000,434808,7009,121,...,0,0,0,0,0
3,1139251,38500,1026470,332,121,...,0,0,0,0,0
4,1139253,11000,1057373,17311,121,...,0,0,0,0,0


### Split into train and test

In [24]:
dfc = df_cats.iloc[0:50].copy(deep=True)

(nrows, ncols) = df_cats.shape
print('Data rows: {}'.format(nrows))

Data rows: 401125


In [25]:
#df_proc.iloc[[1,2,100]]

In [26]:
np.random.seed(seed=23)
x = dfc.index.values
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [27]:
x = np.random.permutation(x)
x

array([14, 18, 17, 32, 13, 33, 20,  8, 36, 28, 29, 10, 44,  3, 22, 48, 30,
       35, 24, 23,  4, 42,  7,  1, 49, 41, 45, 15, 46, 16, 34, 37,  0,  5,
       21, 11,  2, 43,  6, 25, 26, 39, 27, 12, 31, 47,  9, 40, 38, 19])

In [28]:
N = 10
y = np.sort(x[:10])
y

array([ 8, 13, 14, 17, 18, 20, 28, 32, 33, 36])

In [29]:
dfc.iloc[y]

,SalesID,SalePrice,MachineID,ModelID,datasource,...,Backhoe_Mounting_category,Blade_Type_category,Travel_Controls_category,Differential_Type_category,Steering_Controls_category
8,1139272,21500,1036251,36003,121,...,0,0,0,0,0
13,1139284,30500,1068082,3542,121,...,0,0,0,0,0
14,1139290,28000,1058450,5162,121,...,0,0,0,0,0
17,1139299,9500,1002713,21442,121,...,0,0,0,0,0
18,1139301,12500,125790,7040,121,...,0,0,0,0,0
20,1139311,41000,1014135,8867,121,...,0,0,0,0,0
28,1139357,46000,44800,19167,121,...,0,0,0,0,0
32,1139367,31500,1036100,9109,121,...,0,0,0,4,2
33,1139369,14000,1050658,1918,121,...,1,5,6,0,0
36,1139390,8250,1011114,7110,121,...,0,0,0,0,0
